In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
import time
import cv2
import numpy as np
import copy
import torch
import torch.nn as nn
from torchvision import transforms  
import imutils     
import ctypes
from cv2 import aruco 
import math 
# import RRDBNet_arch as arch     
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights

In [ ]:
from task_2a import detect_ArUco_details

In [ ]:
def distance(ar1, ar2):
    c1 = ar1[0]
    x1 = c1[0], y1 = c1[1]
    c2 = ar2[0]
    x2 = c2[0], y2 = c2[1]

    width = x2-x1
    height = y2-y1
    dist = math.sqrt(pow(width, 2) + pow(height, 2))
    return dist
    

In [2]:
path = [23, 24, 22, 49, 34, 33, 35, 38, 37, 36, 8, 12, 9, 11, 29, 28, 27, 26, 25, 22, 24, 23]
bot_marker = 69

In [ ]:
def locate():
    pass

In [3]:
# Get screen size
user32 = ctypes.windll.user32
screen_width = user32.GetSystemMetrics(0)

# Open the camera
cap = cv2.VideoCapture(0)

# Check if the camera is opened successfully
if not cap.isOpened():
    print("Unable to open the camera")
    exit()

# Flag to check if the picture has been taken
curr_node = 0
# Read and display frames from the camera
while curr_node < len(path) - 1:
    ret, frame = cap.read()

    if not ret:
        print("Error reading frame from the camera")
        break

    # Resize the frame to half of the screen width
    new_width = screen_width // 2
    new_height = frame.shape[0] * new_width // frame.shape[1]
    frame = cv2.resize(frame, (new_width, new_height))

    details, corners = detect_ArUco_details(frame)

    if distance(details[bot_marker], details[curr_node+1]) < distance(details[bot_marker], details[curr_node]):
        curr_node += 1
        locate(path[curr_node])

    cv2.imshow("Live Feed", frame)

    # Move the window to the left
    cv2.moveWindow("Live Feed", 0, 0)

    # Take a picture after 3 seconds
    # time.sleep(3)
    # cv2.imwrite('eval.jpg', frame)
    picture_taken = True

cap.release()
cv2.waitKey(0)
cv2.destroyAllWindows()

: 

In [ ]:
# import cv2

# # Create a video capture object and pass 0 as the argument to access the webcam
# cap = cv2.VideoCapture(0)

# # Check if the webcam is opened correctly
# if not cap.isOpened():
#     raise IOError("Cannot open webcam")

# # Loop until the user presses the ESC key
# while True:
#     # Read the current frame from the webcam
#     ret, frame = cap.read()

#     # Resize the frame to 640x480 pixels
#     frame = cv2.resize(frame, (640, 480))

#     # Display the frame in a window named 'Webcam'
#     cv2.imshow('Webcam', frame)

#     # Wait for 10 ms and check if the user pressed the ESC key
#     c = cv2.waitKey(10)
#     if c == 27:
#         break

# # Release the video capture object and close the window
# cap.release()
# cv2.destroyAllWindows()
